In [11]:
import numpy as np
import pandas as pd
import json
import folium

In [12]:
cd_ = "C:\Myfolder\KDT\datas"

crime = pd.read_csv(cd_ + "/crime_in_Seoul.csv",
                 thousands=",",
                 index_col="관서명",
                 encoding="cp949")

with open(cd_ + "\police_station_info.json","r",encoding="cp949") as f:
    data_polices = json.load(f)

In [13]:
police_info_list = []
#주소, 위도, 경도
for i in data_polices:
    dict_ = {}
    dict_["police_name"] = i["police_name"]
    dict_["address"] = i["formatted_address"]
    dict_["lat"] = i["geometry"]["location"]["lat"]
    dict_["lng"] = i["geometry"]["location"]["lng"]
    dict_["gu"] = i["formatted_address"].split(" ")[2]

    police_info_list.append(dict_)

police_info_df = pd.DataFrame(police_info_list)
police_info_df.loc[6, "gu"] = "노원구"

In [14]:
station_name = []

for name in crime.index:
    station_name.append("서울" + str(name[:-1] + "경찰서"))

crime["police_name"] = station_name

indexes_from_json = [item.replace("서울","").replace("경찰","") for item in police_info_df["police_name"]]
police_info_df.index = indexes_from_json

fin_police_info = pd.merge(left=crime,
                          right=police_info_df,
                          how="outer", # inner 시 공통이 없으면 날아감, outer 는 공통이든 뭐든 전부 
                          on=None) # on 은 합칠 컬럼 기준 None, left_on, right_on

police_info_df = pd.DataFrame(police_info_list)
police_info_df.loc[6, "gu"] = "노원구"

In [15]:
crime.head(3)

,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,police_name
관서명,,,,,,,,,,,
중부서,2,2,3,2,105,65,1395,477,1355,1170,서울중부경찰서
종로서,3,3,6,5,115,98,1070,413,1278,1070,서울종로경찰서
남대문서,1,0,6,4,65,46,1153,382,869,794,서울남대문경찰서


### 1. 강력범죄 검거 건수와 발생 건수의 비율을 계산하여 "arrest_occurs_ratio" 열을 추가하시오. 만약 `NaN`이 있다면 0으로 채우시오.

In [16]:
crime_occurs = crime.columns[:-1:2] # 전체를 두칸 뛰어 가져 오는데, 마지막 컬럼은 police_name 이어서 하나빼고 가져온다.
crime_arrest = crime.columns[1::2] # 첫 인덱스인 0 이후부터 2칸띄어 가져오기

#새로운 시리즈 데이터
arrest_occurs_ratio = fin_police_info[crime_arrest].sum(axis=1) / fin_police_info[crime_occurs].sum(axis=1)
fin_police_info["arrest_occurs_ratio"] = arrest_occurs_ratio
fin_police_info.fillna(value=0, inplace=True)

fin_police_info.tail(3)

,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,police_name,address,lat,lng,gu,arrest_occurs_ratio
28,10.0,7.0,6.0,6.0,149.0,124.0,1439.0,666.0,1819.0,1559.0,서울수서경찰서,대한민국 서울특별시 강남구 개포로 617,37.493490,127.077212,강남구,0.690038
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,서울금천경찰서,대한민국 서울특별시 금천구 시흥대로73길 50,37.456813,126.896806,금천구,0.000000
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,서울은평경찰서,대한민국 서울특별시 은평구 연서로 365,37.628020,126.928790,은평구,0.000000


#### 해답

In [26]:
#새로운 시리즈 데이터
fin_police_info["arrest_occurs_ratio"] = fin_police_info[crime_arrest].sum(axis=1)\
                                         / fin_police_info[crime_occurs].sum(axis=1).fillna(0.0)

fin_police_info.tail(3)

,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,police_name,address,lat,lng,gu,arrest_occurs_ratio
28,10.0,7.0,6.0,6.0,149.0,124.0,1439.0,666.0,1819.0,1559.0,서울수서경찰서,대한민국 서울특별시 강남구 개포로 617,37.493490,127.077212,강남구,0.690038
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,서울금천경찰서,대한민국 서울특별시 금천구 시흥대로73길 50,37.456813,126.896806,금천구,NaN
30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,서울은평경찰서,대한민국 서울특별시 은평구 연서로 365,37.628020,126.928790,은평구,NaN


###  2. "arrest_occurs_ratio"가 가장 큰 경찰서를 찾으시오.

In [17]:
max_ratio = fin_police_info[fin_police_info["arrest_occurs_ratio"] == fin_police_info["arrest_occurs_ratio"].max()]["police_name"]
max_ratio

18    서울강서경찰서
Name: police_name, dtype: object

##### 수정

In [18]:
max_ratio = fin_police_info["arrest_occurs_ratio"].idxmax()
fin_police_info.iloc[max_ratio][["police_name"]]

police_name    서울강서경찰서
Name: 18, dtype: object

###  3. "arrest_occurs_ratio"가 가장 작은 경찰서를 찾으시오.(단, 0은 제외한다.)

In [19]:
over_zero = fin_police_info["arrest_occurs_ratio"][fin_police_info["arrest_occurs_ratio"] > 0]

min_ratio_idx = over_zero[over_zero == over_zero.min()].index
fin_police_info.iloc[min_ratio_idx]["police_name"]

26    서울방배경찰서
Name: police_name, dtype: object

##### 수정

In [20]:
over_zero = fin_police_info["arrest_occurs_ratio"][fin_police_info["arrest_occurs_ratio"] > 0]

min_ratio_idx = over_zero.argmin()
fin_police_info.iloc[min_ratio_idx]["police_name"]

'서울방배경찰서'

###  4. crime_police_outer에서 구의 개수를 세시오. 

In [21]:
gus = len(set([gus for gus in fin_police_info["gu"]]))
print(f"'구의 개수':{gus}")

'구의 개수':25


#### 수정(해답)

In [28]:
gu_s = len(fin_police_info["gu"].unique())
print(f"'구의 개수':{gu_s}")

fin_police_info["gu"].value_counts()

'구의 개수':25


중구      2
강남구     2
성북구     2
서초구     2
은평구     2
종로구     2
도봉구     1
노원구     1
송파구     1
양천구     1
구로구     1
강동구     1
강서구     1
관악구     1
강북구     1
중랑구     1
광진구     1
동작구     1
성동구     1
영등포구    1
마포구     1
동대문구    1
용산구     1
서대문구    1
금천구     1
Name: gu, dtype: int64

###  5. 구별로 "occurs_total"과 "arrest_total"를 groupby를 이용하여 계산하시오.

In [29]:
total_dict = {}

for i, j in fin_police_info.groupby(by="gu"):
    total_dict[i] = [j[crime_occurs].sum(axis=1).sum(),j[crime_arrest].sum(axis=1).sum()]

grouped_df = pd.DataFrame(total_dict).T
grouped_df.columns = ["total_occurs", "total_arrest"]

grouped_df.head(10)

,total_occurs,total_arrest
강남구,8617.0,5732.0
강동구,5244.0,3171.0
강북구,4257.0,3113.0
강서구,5585.0,4190.0
관악구,6345.0,3712.0
광진구,5909.0,3707.0
구로구,5646.0,3502.0
금천구,0.0,0.0
노원구,5130.0,3268.0
도봉구,2664.0,1900.0


#### 해답 (다시 살펴보기)

In [36]:
grouped_df = fin_police_info.groupby(["gu"]).agg(["sum"])
grouped_df

,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,police_name,address,lat,lng,arrest_occurs_ratio
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
gu,,,,,,,,,,,,,,,
강남구,13.0,10.0,21.0,18.0,449.0,349.0,3850.0,1650.0,4284.0,3705.0,서울강남경찰서서울수서경찰서,대한민국 서울특별시 강남구 테헤란로114길 11대한민국 서울특별시 강남구 개포로 617,75.002925,254.144170,1.338864
강동구,4.0,3.0,6.0,8.0,156.0,123.0,2366.0,789.0,2712.0,2248.0,서울강동경찰서,대한민국 서울특별시 강동구 성내로 57,37.528511,127.126822,0.604691
강북구,7.0,8.0,14.0,13.0,153.0,126.0,1434.0,618.0,2649.0,2348.0,서울강북경찰서,대한민국 서울특별시 강북구 오패산로 406,37.637197,127.027305,0.731266
강서구,7.0,8.0,13.0,13.0,262.0,191.0,2096.0,1260.0,3207.0,2718.0,서울강서경찰서,대한민국 서울특별시 강서구 화곡로 308,37.551362,126.850280,0.750224
관악구,9.0,8.0,12.0,14.0,320.0,221.0,2706.0,827.0,3298.0,2642.0,서울관악경찰서,대한민국 서울특별시 관악구 관악로5길 33,37.474395,126.951349,0.585028
광진구,4.0,4.0,14.0,26.0,240.0,220.0,3026.0,1277.0,2625.0,2180.0,서울광진경찰서,대한민국 서울특별시 광진구 자양로 167,37.542823,127.083839,0.627348
구로구,8.0,6.0,15.0,11.0,281.0,164.0,2335.0,889.0,3007.0,2432.0,서울구로경찰서,대한민국 서울특별시 구로구 새말로 97 신도림테크노마트 5층,37.507442,126.890224,0.620262
금천구,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,서울금천경찰서,대한민국 서울특별시 금천구 시흥대로73길 50,37.456813,126.896806,0.000000


###  6. "금천구"의 "occurs_total"과 "arrest_total"를 평균으로 채우시오.

In [24]:
grouped_df[grouped_df == 0] = np.nan
grouped_df.fillna(value = grouped_df.mean(),inplace=True)
grouped_df = grouped_df.round()
grouped_df.loc["금천구"]

total_occurs    4998.0
total_arrest    3193.0
Name: 금천구, dtype: float64

### 7. "seoul_geo.json"을 이용하여 구별로 "arrest_total"을 반영한 Choropleth을 그리시오.

In [25]:
map = folium.Map(location = [37.5642135, 127.0016985],
                     zoom_start = 11)
with open("C:/Myfolder/KDT/datas/seoul_geo.json", encoding="utf-8") as f:
    geo_json = json.load(f)

folium.Choropleth(geo_data = geo_json,
                  data = grouped_df.total_arrest,
                  fill_color="PuBu",
                  key_on="feature.id",).add_to(map)

map